<a href="https://colab.research.google.com/github/wsokal/gold_buying_app/blob/main/holdemstarters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install treys


In [2]:
import random
import sqlite3
from treys import Deck, Evaluator, Card

# --- SETUP: SQLite Database for Recording Statistics ---

# Connect to (or create) a local SQLite database file.
conn = sqlite3.connect('texas_holdem_stats.db')
cursor = conn.cursor()

# Create a table to store the starting hand statistics.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS starter_hand_stats (
        hand_category TEXT PRIMARY KEY,
        wins REAL,
        total INTEGER
    )
''')
conn.commit()


# --- HELPER FUNCTIONS ---

def classify_starter_hand(card1, card2):
    """
    Convert two hole cards into one of 169 starting hand categories.
    Uses card rank (e.g. 'A', 'K', 'Q', etc.) and suitedness.
    For a pair, returns e.g., "AA".
    For two different cards, returns e.g., "AKs" for suited or "AKo" for offsuit.
    """
    # Convert card integers into their string representation, e.g., "Ah", "Kd", etc.
    card_str1 = Card.int_to_str(card1)  # e.g., "Ah"
    card_str2 = Card.int_to_str(card2)  # e.g., "Kd"

    rank1, suit1 = card_str1[0], card_str1[1]
    rank2, suit2 = card_str2[0], card_str2[1]

    # Define the rank order for comparison.
    ranks_order = "23456789TJQKA"

    # Order the ranks so that the highest appears first.
    if ranks_order.index(rank1) < ranks_order.index(rank2):
        rank1, rank2 = rank2, rank1
        suit1, suit2 = suit2, suit1

    if rank1 == rank2:
        # Pocket pair: no suitedness marker.
        return rank1 + rank2
    else:
        # Non-paired hands: add 's' if suited, otherwise 'o'
        suited = 's' if suit1 == suit2 else 'o'
        return rank1 + rank2 + suited


def update_database(starting_hands, winners):
    """
    For each player’s starting hand category, update the running win/total counts.
    If multiple players tie for the win, split the win credit equally.
    """
    for i, hand_category in enumerate(starting_hands):
        # Determine win fraction: if the player is in the list of winners, they get a share of 1.
        win_fraction = 1.0 / len(winners) if i in winners else 0.0

        # Insert or update the row in the table.
        cursor.execute('''
            INSERT INTO starter_hand_stats (hand_category, wins, total)
            VALUES (?, ?, ?)
            ON CONFLICT(hand_category) DO UPDATE SET
                wins = wins + ?,
                total = total + ?
        ''', (hand_category, win_fraction, 1, win_fraction, 1))
    conn.commit()


# --- SIMULATION FUNCTIONS ---

def simulate_game():
    """
    Simulate a single Texas Hold'em game with 10 players.
    - Every player is dealt 2 hole cards.
    - 5 community cards are dealt.
    - All players stay in (no folding).
    - The best hand wins (ties are split).

    Returns:
        starting_hands: A list (length 10) of starting hand categories (e.g., "AKs", "77", etc.)
        winners: A list of player indices (0-based) that won the game.
    """
    deck = Deck()
    evaluator = Evaluator()

    # Deal hole cards to 10 players.
    players_hole = [deck.draw(2) for _ in range(10)]

    # Deal 5 community cards.
    board = deck.draw(5)

    # Evaluate each player's best hand using the evaluator.
    scores = [evaluator.evaluate(hand, board) for hand in players_hole]
    best_score = min(scores)  # Lower score means a stronger hand.
    winners = [i for i, score in enumerate(scores) if score == best_score]

    # Classify each player's starting hand.
    starting_hands = [classify_starter_hand(hand[0], hand[1]) for hand in players_hole]

    return starting_hands, winners


def run_simulations(num_iterations=10000):
    """
    Run a number of simulated games and update the database with results.
    """
    for i in range(1, num_iterations + 1):
        starting_hands, winners = simulate_game()
        update_database(starting_hands, winners)

        if i % 1000 == 0:
            print(f"Completed {i} iterations")


# --- MAIN ---

if __name__ == "__main__":
    total_iterations = 10000  # Change this value as needed.
    run_simulations(total_iterations)

    # Query and print the results.
    cursor.execute("SELECT * FROM starter_hand_stats ORDER BY total DESC")
    results = cursor.fetchall()
    print("\n=== Starter Hand Statistics ===")
    for hand_category, wins, total in results:
        win_rate = wins / total if total > 0 else 0
        print(f"Hand {hand_category}: Wins = {wins:.3f}, Total = {total}, Win Rate = {win_rate:.3f}")

    # Close the database connection.
    conn.close()


Completed 1000 iterations
Completed 2000 iterations
Completed 3000 iterations
Completed 4000 iterations
Completed 5000 iterations
Completed 6000 iterations
Completed 7000 iterations
Completed 8000 iterations
Completed 9000 iterations
Completed 10000 iterations

=== Starter Hand Statistics ===
Hand 43o: Wins = 65.283, Total = 983, Win Rate = 0.066
Hand T7o: Wins = 95.833, Total = 969, Win Rate = 0.099
Hand Q2o: Wins = 62.333, Total = 957, Win Rate = 0.065
Hand T3o: Wins = 60.667, Total = 957, Win Rate = 0.063
Hand J7o: Wins = 85.783, Total = 957, Win Rate = 0.090
Hand 82o: Wins = 44.433, Total = 955, Win Rate = 0.047
Hand J6o: Wins = 61.417, Total = 953, Win Rate = 0.064
Hand Q4o: Wins = 71.517, Total = 951, Win Rate = 0.075
Hand A9o: Wins = 108.183, Total = 950, Win Rate = 0.114
Hand T8o: Wins = 96.283, Total = 950, Win Rate = 0.101
Hand A4o: Wins = 109.500, Total = 947, Win Rate = 0.116
Hand 97o: Wins = 86.850, Total = 947, Win Rate = 0.092
Hand JTo: Wins = 128.683, Total = 944, Win R